In [1]:
!kaggle datasets download -d kssanjaynithish03/retinal-fundus-images

Dataset URL: https://www.kaggle.com/datasets/kssanjaynithish03/retinal-fundus-images
License(s): CC-BY-NC-SA-4.0
100% 2.32G/2.32G [01:06<00:00, 35.5MB/s]
100% 2.32G/2.32G [01:06<00:00, 37.7MB/s]


In [2]:
!unzip retinal-fundus-images.zip

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: Retinal Fundus Images/train/6.Proliferate DR/Proliferate DR23604_left_0_4947.jpeg  
  inflating: Retinal Fundus Images/train/6.Proliferate DR/Proliferate DR23604_left_0_8707.jpeg  
  inflating: Retinal Fundus Images/train/6.Proliferate DR/Proliferate DR23648_right_0_1409.jpeg  
  inflating: Retinal Fundus Images/train/6.Proliferate DR/Proliferate DR23648_right_0_4748.jpeg  
  inflating: Retinal Fundus Images/train/6.Proliferate DR/Proliferate DR23692_left_0_5295.jpeg  
  inflating: Retinal Fundus Images/train/6.Proliferate DR/Proliferate DR23692_left_0_8794.jpeg  
  inflating: Retinal Fundus Images/train/6.Proliferate DR/Proliferate DR23692_right_0_2860.jpeg  
  inflating: Retinal Fundus Images/train/6.Proliferate DR/Proliferate DR23692_right_0_6729.jpeg  
  inflating: Retinal Fundus Images/train/6.Proliferate DR/Proliferate DR23736_left_0_6286.jpeg  
  inflating: Retinal Fundus Images/train/6.Proliferate DR/Prolifer

In [ ]:
# !pip install kaggle efficientnet_pytorch

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.5 MB/s eta 0:00:00
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=21c124f154ef1528e88de2c2615b554a660ae1d4b17253c3d

In [3]:
# Mild DR: Bệnh võng mạc tiểu đường mức độ nhẹ.
# Moderate DR: Bệnh võng mạc tiểu đường mức độ trung bình.
# Severe DR: Bệnh võng mạc tiểu đường mức độ nặng.
# Normal Fundus: Ảnh võng mạc bình thường.
root_name = ['3.Mild DR', '4.Moderate DR', '5.Severe DR', '11.Normal Fundus']

In [10]:
import os
import shutil

# Định nghĩa đường dẫn gốc
root_dir = "Retinal Fundus Images"
new_root = "datasets"

# Các tập con cần giữ lại
selected_folders = {
    "3.Mild DR": "2.Mild DR",
    "4.Moderate DR": "3.Moderate DR",
    "5.Severe DR": "4.Severe DR",
    "11.Normal Fundus": "1.Normal Fundus"
}

# Các thư mục con trong datasets
sub_dirs = ["train", "test", "val"]

# Tạo thư mục datasets với cấu trúc mới
for sub_dir in sub_dirs:
    old_path = os.path.join(root_dir, sub_dir)
    new_path = os.path.join(new_root, sub_dir)

    os.makedirs(new_path, exist_ok=True)

    for old_folder, new_folder in selected_folders.items():
        src = os.path.join(old_path, old_folder)
        dst = os.path.join(new_path, new_folder)

        if os.path.exists(src):
            shutil.copytree(src, dst)
        else:
            print(f"Folder {src} does not exist!")

print("Dataset reorganization complete!")


Dataset reorganization complete!


Import library

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
from PIL import Image

# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [12]:
# Hàm huấn luyện mô hình
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = 100 * correct / total
        val_acc = evaluate_model(model, val_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")

    print("Training Complete!")

# Hàm đánh giá mô hình trên validation/test
def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

In [16]:
# Định nghĩa đường dẫn thư mục
data_dir = "datasetss/"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")

# Tiền xử lý ảnh
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize ảnh về 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Chuẩn hóa theo ImageNet
])

In [17]:
# Định nghĩa các tham số
BATCH_SIZE = 32
NUM_CLASSES = 4  # 4 class bạn đã chọn
EPOCHS = 10
LEARNING_RATE = 0.001

In [18]:
# Load dataset
train_dataset = ImageFolder(root=train_dir, transform=transform)
val_dataset = ImageFolder(root=val_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)

# Tạo DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [19]:
# Tải ResNet50 Pre-trained
model = models.resnet50(pretrained=True)

# Thay đổi Fully Connected Layer cuối cùng để phù hợp với 4 lớp
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)

# Đưa mô hình lên GPU (nếu có)
model = model.to(device)

# Định nghĩa hàm mất mát và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 178MB/s]


In [20]:
# Huấn luyện mô hình
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=EPOCHS)

Epoch [1/10], Loss: 0.4762, Train Acc: 77.22%, Val Acc: 80.85%
Epoch [2/10], Loss: 0.3907, Train Acc: 80.35%, Val Acc: 83.40%
Epoch [3/10], Loss: 0.3624, Train Acc: 82.00%, Val Acc: 83.40%
Epoch [4/10], Loss: 0.3479, Train Acc: 82.57%, Val Acc: 80.85%
Epoch [5/10], Loss: 0.3424, Train Acc: 83.05%, Val Acc: 82.55%
Epoch [6/10], Loss: 0.3326, Train Acc: 83.67%, Val Acc: 84.68%
Epoch [7/10], Loss: 0.3022, Train Acc: 85.34%, Val Acc: 79.15%
Epoch [8/10], Loss: 0.2957, Train Acc: 85.81%, Val Acc: 83.40%
Epoch [9/10], Loss: 0.2783, Train Acc: 87.08%, Val Acc: 82.55%
Epoch [10/10], Loss: 0.2590, Train Acc: 87.82%, Val Acc: 82.98%
Training Complete!


In [21]:
# Đánh giá trên tập test
test_acc = evaluate_model(model, test_loader)
print(f"Test Accuracy: {test_acc:.2f}%")

# Lưu mô hình
torch.save(model.state_dict(), "resnet50_model.pth")
print("Model saved as resnet50_model.pth")

Test Accuracy: 89.07%
Model saved as resnet50_model.pth


In [24]:
# Load the saved model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4) # Assuming 4 output classes
model.load_state_dict(torch.load("resnet50_model.pth", map_location=device)) # Load weights
model.to(device)
model.eval()

# Define image transformations (must match training transformations)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def predict_image(image_path):
    try:
        image = Image.open(image_path).convert("RGB")  # Ensure the image is RGB
        image = transform(image).unsqueeze(0).to(device) # Add batch dimension

        with torch.no_grad():
            output = model(image)
            _, predicted_class = torch.max(output, 1)

        class_names = ["1.Normal Fundus", "2.Mild DR", "3.Moderate DR", "4.Severe DR"] # Adjust class names
        predicted_label = class_names[predicted_class.item()]
        return predicted_label
    except FileNotFoundError:
        return "Image file not found."
    except Exception as e:
        return f"An error occurred: {e}"



<ipython-input-24-2aeef2d9b3e6>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("resnet50_model.pth", map_location=device)) # Load weights


In [26]:
# Example usage
image_path = "/content/datasetss/test/2.Mild DR/1666_right.jpeg"  # Replace with the actual path to your image
prediction = predict_image(image_path)
print(f"The predicted class for {image_path} is: {prediction}")

The predicted class for /content/datasetss/test/2.Mild DR/1666_right.jpeg is: 2.Mild DR
